<a href="https://colab.research.google.com/github/Chuck2Win/attention/blob/Attention3_4_10/attention_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from gensim.models import Word2Vec
import nltk
import os
from google.colab import files
from google.colab import drive
from matplotlib import pyplot as plt

In [3]:
# google drive와 연동
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
print(os.getcwd())
print(os.listdir('./')) # 현재 directory에 있는 모든 녀석들,..
# 기존 경로 + 새로운 폴더 이름으로 새로운 경로 만들기
current=os.getcwd()
path=os.path.join(current,'gdrive/My Drive/attention')
os.chdir(path)
print(os.getcwd())
# 새로운 directory 만들기
#os.mkdir(path)
# 새로운 경로로 바꾸기
#os.chdir(path)
print(os.listdir('./'))

/content
['.config', 'gdrive', 'sample_data']
/content/gdrive/My Drive/attention
['eng-fra.txt']


In [5]:
# drive에 파일 올리기
files.upload()

KeyboardInterrupt: ignored

In [0]:
import pandas as pd
from pandas import DataFrame as df
data=pd.read_csv('./eng-fra.txt',sep='\t',names=['english','french'],header=None) # column명이 없는 경우 column 명을 지정하고, 불러온다.

In [10]:
data.head()

,english,french
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !
3,Wow!,Ça alors !
4,Fire!,Au feu !


In [0]:
# nltk의 word tokenize를 활용해보고
# 일단 lower를 진행..

In [12]:
# tokenize하기
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# word_tokenize(sentene) <- dataframe이 들어가면 안된다...
data['tokenized_eng']=data['english'].str.lower().apply(lambda i : word_tokenize(i,language='english'))
data['tokenized_french']=data['french'].str.lower().apply(lambda i : word_tokenize(i,language='french'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
# torch text를 활용않고, vocabulary 만들어서 진행하자....
print(max(data['tokenized_eng'].apply(lambda i : len(i)))) # 51 
print(max(data['tokenized_french'].apply(lambda i : len(i)))) # 58

51
58


In [0]:
# 만약 다 손수 한다면...
def make_vocab(sentences,min_count=3):
    # sentences는 pandas 형태일 것이다.
    result={}
    
    for sentence in sentences:
        for vocab in sentence:
            if vocab not in result.keys():
                result[vocab]=1
            else:
                result[vocab]+=1
    #<- Counter 역할을 한 것이고.
    Result={key:_+4 for _,key in enumerate(result.keys()) if result[key]>=min_count}
    Result['<unk>']=0
    Result['<pad>']=1
    Result['<sos>']=2
    Result['<eos>']=3
    return Result
english_index=make_vocab(data.tokenized_eng)
french_index=make_vocab(data.tokenized_french)

In [0]:
# tokenized->vectorization # ㅋㅋㅋ 결국 padding 진행
def vectorizing(sentence,word2index,max_len=64):
    result=[2]
    for i in sentence:
        if i in word2index.keys():
            result.append(word2index[i])
        else:
            result.append(word2index['<unk>'])
    result.append(3)
    l=max_len-len(sentence)
    result.extend([1]*l)
    return result
data['vectorized_eng']=data['tokenized_eng'].apply(lambda i : vectorizing(i,english_index))
data['vectorized_fr']=data['tokenized_french'].apply(lambda i : vectorizing(i,french_index))

In [125]:
print(english_index['<sos>']) # 2
print(english_index['<eos>']) # 3
print(len(english_index.keys()))

2
3
7170


In [86]:
# vectorized_eng -> vectorized_fr
print(data.vectorized_eng.head())
print(data.vectorized_fr.head())

0    [2, 4, 5, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1    [2, 6, 7, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
2    [2, 6, 7, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
3    [2, 8, 7, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
4    [2, 9, 7, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
Name: vectorized_eng, dtype: object
0    [2, 4, 5, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1    [2, 6, 5, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
2    [2, 7, 5, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
3    [2, 8, 9, 5, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
4    [2, 10, 11, 5, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...
Name: vectorized_fr, dtype: object
66


In [0]:
### 여기까진 데이터 전처리 ###

In [0]:
## torch.utils.data.DataLoader을 활용할 것인데
x=torch.tensor(data['vectorized_eng'].to_list())
y=torch.tensor(data['vectorized_fr'].to_list())
X=torch.utils.data.TensorDataset(x,y)
train_loader=torch.utils.data.DataLoader(X,batch_size=64,drop_last=True)

In [0]:
batch=next(iter(train_loader))

In [153]:
print(batch[0].shape) # 4,66
print(batch[0]) # english
print(batch[1].shape) # 64,66
print(batch[1]) # french

torch.Size([64, 66])
tensor([[ 2,  4,  5,  ...,  1,  1,  1],
        [ 2,  6,  7,  ...,  1,  1,  1],
        [ 2,  6,  7,  ...,  1,  1,  1],
        ...,
        [ 2, 46, 47,  ...,  1,  1,  1],
        [ 2, 46, 47,  ...,  1,  1,  1],
        [ 2, 46, 48,  ...,  1,  1,  1]])
torch.Size([64, 66])
tensor([[ 2,  4,  5,  ...,  1,  1,  1],
        [ 2,  6,  5,  ...,  1,  1,  1],
        [ 2,  7,  5,  ...,  1,  1,  1],
        ...,
        [ 2, 92, 91,  ...,  1,  1,  1],
        [ 2, 92, 93,  ...,  1,  1,  1],
        [ 2, 90, 94,  ...,  1,  1,  1]])


In [0]:
# positional encoding을 해줘야한다.(실제 모델에서는 pretrained 모델을 활용한다고 했다)
def positional_encoding(input):
    # input shape : [n,seq_len,embedding_dim]
    def for_insert(input):
        # input shape : n, seq_len, embedding_dim
        for i in range(input.shape[1]):
            for j in range(input.shape[2]):
                input[:,i,j]=torch.tensor(i/(10000)**(j/input.shape[2]))
        return input
    pe=for_insert(input)
    pe[:,:,0::2]=torch.sin(pe[:,:,0::2])
    pe[:,:,1::2]=torch.cos(pe[:,:,0::2])
    return pe

In [0]:
def self_attention(input,Mask=False):

    # scaled dot product attention
    # input shape : [n,seq_len,embedding_dim]
    Result=[]
    mask=torch.ones((input.shape[1],input.shape[1])).triu(diagonal=1).bool() # seq_len,seq_len
    for ba in range(input.shape[0]):
        Q=input[ba]
        K=Q
        V=Q
        T=Q.matmul(K.T)/torch.sqrt(torch.tensor(Q.shape[-1]).float())
        if Mask:
            T.masked_fill_(mask,1e-7)
        weight=F.softmax(T,dim=-1)
        # weight : seq_len, seq_len
        attention_score=weight.matmul(V)
        # attention score : seq_len,embedding_dim
        Result.append(attention_score)
    return torch.stack(Result,0)# Result : n, seq_len, embedding_dim

In [0]:
def attention(encoder_input,decoder_input):
    # scaled dot product attention
    # encoder_input shape : [n,encoder_seq_len,encoder_embedding_dim]
    # decoder_input shape : [n,decoder_seq_len,decoder_embedding_dim]
    Result=[]
    for ba in range(encoder_input.shape[0]):
        Q=decoder_input[ba]
        K=encoder_input[ba]
        V=encoder_input[ba]
        T=Q.matmul(K.T)/torch.sqrt(torch.tensor(Q.shape[-1]).float())
        weight=F.softmax(T,dim=-1)
        # weight : seq_len, seq_len
        attention_score=weight.matmul(V)
        # attention score : seq_len,embedding_dim
        Result.append(attention_score)
    return torch.stack(Result,0)# Result : n, seq_len, embedding_dim

In [0]:
class attention_encoder(nn.Module): # 완료
    def __init__(self,n_vocab,seq_len,embedding_dim):
        super().__init__()
        self.embedding=nn.Embedding(n_vocab,embedding_dim)
        self.embedding.weight.required_grad=False
        self.embedding.weight
        self.fc=nn.Sequential(nn.Linear(embedding_dim,embedding_dim*4),nn.ReLU(),nn.Linear(embedding_dim*4,embedding_dim))
        self.batch1=nn.BatchNorm1d(seq_len)
        self.batch2=nn.BatchNorm1d(seq_len)
    def forward(self,input):
        # input shape : [n,seq_len] # n batch size
        x=self.embedding(input) # x : [n,seq_len,embedding_dim]
        self.pe=positional_encoding(x)
        x=self_attention(x)
        x=self.batch1(x+self.pe)
        # x=layer_normalization(x+x_) # residual block
        x=self.batch2(self.fc(x)+x)
        return x

In [0]:
# attention encoder example
model_encoder=attention_encoder(7170,66,300)
encoder_output=model_encoder(batch[0])

In [0]:
class attention_decoder(nn.Module): # encoder와의 차이점 - masking을 해줘야 함. 
    def __init__(self,n_vocab,seq_len,embedding_dim):
        super().__init__()
        self.embedding=nn.Embedding(n_vocab,embedding_dim)
        self.batch1=nn.BatchNorm1d(seq_len) # 여기에 seq_len이 들어가야한다.
        self.fc1=nn.Sequential(nn.Linear(embedding_dim,embedding_dim*4),nn.ReLU(),nn.Linear(embedding_dim*4,embedding_dim))
        self.batch2=nn.BatchNorm1d(seq_len)
        self.batch3=nn.BatchNorm1d(seq_len)
        self.fc2=nn.Sequential(nn.Linear(embedding_dim,embedding_dim*4),nn.ReLU(),nn.Linear(embedding_dim*4,embedding_dim))
    def forward(self,input,encoder_output):
        # input shape : [n,seq_len] # n batch size
        # encoder_output : [n,encoder_seq_len,encoder_embedding_dim]
        
        #1. decoder_self_attention
        x=self.embedding(input) # x : [n,decoder_seq_len,decoder_embedding_dim]
        self.pe=positional_encoding(x)
        x=self_attention(x,True)
        x=self.batch1(x+self.pe)
        # x=layer_normalization(x+x_) # residual block
        x=self.batch2(self.fc1(x)+x)
        # x shape : [n,decoder_seq_len,decoder_embedding_dim]
        
        #2. decoder_encoder_attention
        output=attention(encoder_output,x)
        x=self.batch3(output+x)

        #3. fc2 layer
        x=self.batch3(self.fc2(x)+x)

        #4. 
        return output


In [160]:
model_decoder=attention_decoder(12238,66,300)
# model_encoder=attention_encoder(7170,66,300)
decoder_output=model_decoder(batch[1],encoder_output)

print(decoder_output.shape)

torch.Size([64, 66, 300])


In [0]:
# pytorch를 위한 dataset 구성하기
# x=torch.tensor(data['vectorized_eng'].to_numpy())
# https://discuss.pytorch.org/t/dataloader-for-various-length-of-data/6418
# dataset_eng=torch.utils.data.TensorDataset(x)
# 에러.. 어차피 TensorDataset을 만드려면,, 격자 구조가 되야되네..

In [0]:
# x=data.vectorized_eng.to_numpy()
# eng_dataloader=torch.utils.data.DataLoader(x,batch_size=1,drop_last=True)
# batch=next(iter(eng_dataloader))
# print(batch)
# https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html 참조
# 사실상 padding을 해서 torch.utils.data.TensorDataset()으로 만들어서 DataLoader로 진행하는 것이 깔끔은 하다.
# 하지만 그런식으로 하기 싫을 경우도 대비해서 한번 배워보자
# # torch.utils.data.DataLoader()
# def batch_maker(batch):
#     batch=torch.tensor(batch)
#     return batch
# loader=torch.utils.data.DataLoader(x,2,collate_fn=batch_maker,shuffle=True) # collate function은 단순히 mini batch로 묶어주기 전에, 각각의 data에 대해서 어떻게 처리하냐에 대한 것을 알려주네...

In [71]:
# https://paul-hyun.github.io/transformer-01/ 참고
input=torch.tensor([[3091, 3604,  206, 3958, 3760, 3590,    0,    0],
        [ 212, 3605,   53, 3832, 3596, 3682, 3760, 3590]]
        )
input.eq(0) # input 중에서 0이랑 equal한 녀석을 말하는 것이구나,
print(input.eq(0))
attn_mask=input.eq(0).unsqueeze(1).expand(2,8,8)
scores=torch.tensor([[31.3476, -0.1451, -1.4832, -2.8843, -1.2542,  0.8314, -5.7174, -5.7174],
        [-0.1451, 27.8460,  4.9304,  0.8781,  1.5047,  1.6372,  0.5585,  0.5585],
        [-1.4832,  4.9304, 26.8313,  3.9618,  2.1587,  3.5587,  3.0447,  3.0447],
        [-2.8843,  0.8781,  3.9618, 27.9037,  1.6892,  4.0453,  4.7429,  4.7429],
        [-1.2542,  1.5047,  2.1587,  1.6892, 32.0763,  4.0136, -1.1263, -1.1263],
        [ 0.8314,  1.6372,  3.5587,  4.0453,  4.0136, 38.0982, -2.7216, -2.7216],
        [-5.7174,  0.5585,  3.0447,  4.7429, -1.1263, -2.7216, 56.9788, 56.9788],


        [-5.7174,  0.5585,  3.0447,  4.7429, -1.1263, -2.7216, 56.9788, 56.9788]])
scores=scores.expand(2,8,8)
scores.masked_fill_(attn_mask,0)
seq=torch.randn((1,10))
subsequent_mask = torch.ones_like(seq).unsqueeze(-1).expand(seq.size(0), seq.size(1), seq.size(1))
print(subsequent_mask)
subsequent_mask = subsequent_mask.triu(diagonal=1) # upper triangular part of a matrix(2-D), diagonal=1은 diagonal에서 위로 1칸 부터 1을 채운다는 것
print(subsequent_mask)
subsequent_mask = subsequent_mask.triu(diagonal=2) # diagonal=2은 diagonal에서  위로 2칸까지 1을 채운다는 것
print(subsequent_mask)

tensor([[False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False, False]])


In [124]:
# self_attention_example 

input = torch.randn((2,3,10)) # 2개의 문장이고, 문장의 길이 3, 차원은 10
output,T=self_attention(input,True)
output[0]

tensor([ 1.1870,  0.2906, -0.8740, -0.9634, -0.4164, -0.9439, -1.5815, -0.1751,
        -0.8899,  0.7945])